##### Note to self: Chatbot_ResearchPaper is the conda env used

In [2]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv  ## for storing and retreiving GROQ api key
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key = groq_api_key,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
def get_response(prompt):
    response = llm.invoke(prompt)
    print(response.content)

In [4]:
get_response("How to create a chatbot?")

Creating a chatbot involves several steps, from designing the conversation flow to implementing the natural language processing (NLP) and machine learning (ML) algorithms. Here's a step-by-step guide to help you create a chatbot:

**Step 1: Define the chatbot's purpose and goals**

1. Identify the chatbot's purpose: What problem does it solve? What tasks will it perform?
2. Determine the chatbot's goals: What kind of conversations will it have? What kind of information will it provide?
3. Define the chatbot's personality: Will it be friendly, formal, or humorous?

**Step 2: Choose a platform and tools**

1. Select a platform: Choose a platform that supports chatbot development, such as:
	* Dialogflow (formerly known as API.ai)
	* Microsoft Bot Framework
	* Amazon Lex
	* Rasa
	* ManyChat
2. Choose a programming language: Select a language that you're comfortable with, such as:
	* Python
	* JavaScript
	* Java
	* C#
3. Select a NLP library: Choose a library that provides NLP capabilities,

In [5]:
from langchain.document_loaders import PyPDFLoader

pdf_reader = PyPDFLoader(r"D:\Data Science Projects\Research Paper Summarizer Chatbot\Research-Paper-Summarizer-Chatbot-End-to-End-Gen-AI-Project-using-LLM-and-RAGS\resource\Artificial Intelligence Applications for MEMS-Based Sensors.pdf")
documents = pdf_reader.load()
documents

[Document(metadata={'source': 'D:\\Data Science Projects\\Research Paper Summarizer Chatbot\\Research-Paper-Summarizer-Chatbot-End-to-End-Gen-AI-Project-using-LLM-and-RAGS\\resource\\Artificial Intelligence Applications for MEMS-Based Sensors.pdf', 'page': 0}, page_content='Citation: Podder, I.; Fischl, T.; Bub, U.\nArtiﬁcial Intelligence Applications\nfor MEMS-Based Sensors and\nManufacturing Process Optimization.\nTelecom 2023 ,4, 165–197. https://\ndoi.org/10.3390/telecom4010011\nAcademic Editor: Panagiotis\nSarigiannidis\nReceived: 20 January 2023\nRevised: 1 March 2023\nAccepted: 7 March 2023\nPublished: 9 March 2023\nCopyright: © 2023 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nReview\nArtiﬁcial Intelligence Applications for MEMS-Based Sensors\nand Manufacturing Process Optimization\n

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

## TEXT SPLITTING Mechanisim to split the text into chunks while keeping paragraphs together and retaininng context over pages

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, )
chunks = text_splitter.split_documents(documents)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load Hugging Face model for embedding
model_name = "sentence-transformers/all-mpnet-base-v2"  # A common sentence transformer model
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# # Create embeddings
# embeddings = hf_embeddings.embed_documents(chunks)

# Create FAISS Vector Store
db = FAISS.from_documents(documents=chunks, embedding=hf_embeddings)


c:\Users\giree\anaconda3\envs\Chatbot_ResearchPaper\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:"""
)

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

### ask a query

In [12]:
chat_history = []
query = """What is this paper all about?"""
result = qa(
    {"question": query,
     "chat_history": chat_history}
            )
result["answer"]

'This paper appears to be a review of articles related to the use of artificial intelligence (AI) algorithms in MEMS (Micro-Electro-Mechanical Systems) manufacturing. The paper analyzes various articles and categorizes them based on the AI algorithms used, such as neural networks, deep learning, and machine learning techniques. The paper also discusses the purposes of using AI algorithms in MEMS manufacturing, including classification, regression, fault detection, thermal calibration, and production issue-related tasks.'

In [13]:
chat_history = []
query = """?What all AI techniques are discussed in the given document?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

The document mentions the following AI techniques:

1. Deep Learning (DL)
2. Machine Learning (ML)
3. Natural Language Processing (NLP)
4. Computer Vision
5. Robotics

Additionally, it mentions specific tasks that AI algorithms are used for, such as:

1. Classification
2. Regression
3. Fault detection
4. Thermal calibration
5. Predictive maintenance
